In [ ]:
# Sim A

In [ ]:
import sys
from pathlib import Path
import numpy as np
import torch
from torch.nn.utils import clip_grad_value_
from torch import nn
from sklearn.linear_model import PoissonRegressor
import matplotlib.pyplot as plt

sys.path.append(Path("low_rank_MPLN"))
import low_rank_MPLN

def gen_Sigma(num_features, rho=0.8):
    # AR1 parameters

    # Generate AR1 covariance matrix
    cov_matrix = rho ** np.abs(np.subtract.outer(np.arange(num_features), np.arange(num_features)))

    # Perform eigen decomposition
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

    # Sort eigenvalues and eigenvectors in descending order
    sort_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sort_indices]
    eigenvectors = eigenvectors[:, sort_indices]

    # Calculate V and A
    V = eigenvectors
    a = eigenvalues


    return cov_matrix, V, a

def gen_X(n, num_covariates, intercept=True):
    X = torch.randn((n, num_covariates))
    if intercept and num_covariates > 0:
        X[:,0] = 0
    return X

# Setup generate data
# -------------
n = 5000
num_features = 50
num_covariates = 0
num_latents = 50

Sigma, V, A = gen_Sigma(num_features, rho=0.8)

# Generate the models
model_true = low_rank_MPLN.MPLN(num_latents, num_features, num_covariates, V[:,:num_latents], np.log(A[:num_latents]))
X = gen_X(n, num_covariates, intercept=True)
data = model_true.sample(n, X, poisson=True)


# Train the model
model = low_rank_MPLN.MPLN(num_latents, num_features, num_covariates, V[:,:num_latents])
# For quicker convergence we start at the true value (this is not necessary but we don't need to check convergence in details for each simu) 
fit = model.train_model(X, data, A_init = model_true.a.clone(), verbose=True, num_epochs=1000)


# Save and load
low_rank_MPLN.save_model_fit(sim_name="test_sim", sim_iter=123, sim_path="", model=model, fit=fit)
m, f = low_rank_MPLN.load_model_fit("test_sim", 123)